```python
eosapi.gen_transaction(actions, expiration, reference_block_id)
```

```python
wallet.sign_transaction(trx, public_keys, chain_id)
```

```python
eosapi.push_transaction(trx, compress=0)
```

In [11]:
import json
import init
from uuoskit import uuosapi
from uuoskit import wasmcompiler
init.init()
test_account1 = init.test_account1

In [12]:
a = ['EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL',
    'EOS5gNqDpWm72KzNWFVgBDmybrEtk9qx2ZuAeRbK9iSJMVJ2B8y73']
a.sort()
a

['EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL',
 'EOS5gNqDpWm72KzNWFVgBDmybrEtk9qx2ZuAeRbK9iSJMVJ2B8y73']

In [13]:
a = {
    "account": test_account1,
    "permission": "sayhello",
    "parent": "active",
    "auth": {
        "threshold": 2,
        "keys": [
            {
                "key": "EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL",
                "weight": 1
            },
            {
                "key": "EOS5gNqDpWm72KzNWFVgBDmybrEtk9qx2ZuAeRbK9iSJMVJ2B8y73",
                "weight": 1
            }
        ],
        "accounts": [],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [ ]:
args = {
    'account':test_account1,
    'code':test_account1,
    'type':'sayhello',
    'requirement':'sayhello'
}
r = uuosapi.push_action('eosio', 'linkauth', args, {test_account1:'active'})

In [15]:
uuosapi.get_account(test_account1)

In [16]:
test_account1

'helloworld11'

In [9]:
code = r'''
package main

import "github.com/uuosio/chain"
import "github.com/uuosio/chain/logger"

func main() {
    receiver, _, _ := chain.GetApplyArgs()
    chain.RequireAuth2(receiver, chain.NewName("sayhello"))
    chain.RequireAuth2(receiver, chain.NewName("owner"))    
    logger.Println("hello,world")
}
'''

code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [10]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'sayhello'})
print(r['processed']['action_traces'][0]['console'])

2021-10-10 00:10:07,912 INFO http_client 208 non ok response: 401


ChainException: {
 "code": 401,
 "message": "UnAuthorized",
 "error": {
  "code": 3090003,
  "name": "unsatisfied_authorization",
  "what": "Provided keys, permissions, and delays do not satisfy declared authorizations",
  "details": [
   {
    "message": "transaction declares authority '{\"actor\":\"helloworld11\",\"permission\":\"sayhello\"}', but does not have signatures for it.",
    "file": "authorization_manager.cpp",
    "line_number": 643,
    "method": "get_required_keys"
   }
  ]
 }
}

In [14]:
action = [test_account1, 'sayhello', b'hello,world', {test_account1:'sayhello'}]
info = uuosapi.get_info()
reference_block_id = info['last_irreversible_block_id']
trx = uuosapi.gen_transaction([action], 60*60, reference_block_id)
print(trx)

{"expiration":"2021-01-12T03:50:41","ref_block_num":60532,"ref_block_prefix":2311561874,"max_net_usage_words":0,"max_cpu_usage_ms":0,"delay_sec":0,"context_free_actions":[],"actions":[{"account":"wkpmdjdsztyu","name":"sayhello","authorization":[{"actor":"wkpmdjdsztyu","permission":"sayhello"}],"data":"68656c6c6f2c776f726c64"}],"transaction_extensions":[],"signatures":[],"context_free_data":[]}


In [15]:
uuosapi.get_account(test_account1)['permissions'][2]

{'perm_name': 'sayhello',
 'parent': 'active',
 'required_auth': {'threshold': 2,
  'keys': [{'key': 'EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL',
    'weight': 1},
   {'key': 'EOS5gNqDpWm72KzNWFVgBDmybrEtk9qx2ZuAeRbK9iSJMVJ2B8y73',
    'weight': 1}],
  'accounts': [],
  'waits': []}}

### sign with private key of EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL

In [16]:
import hashlib
import json
public_keys = ['EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL',]
info = uuosapi.get_info()
signed_trx = wallet.sign_transaction(trx, public_keys, info['chain_id'])
sign1 = json.loads(signed_trx)['signatures']
sign1

['SIG_K1_Kg3bwKkiPHu71aCM8dYvwZLn8pxGWWu4uZ9Zv5J8JQtsWmsnz5bMpuLbUNh4T4KtDVxWvKVMoRBFzmEvuqPGLzhAQz1rAP']

In [17]:
import hashlib
info = uuosapi.get_info()
chain_id = bytes.fromhex(info['chain_id'])

packed_trx = uuosapi.pack_transaction(trx)
packed_trx = json.loads(packed_trx)
packed_trx = bytes.fromhex(packed_trx['packed_trx'])

h = hashlib.sha256()
h.update(chain_id)
h.update(packed_trx)
h.update(b'\x00'*32)
wallet.sign_digest(h.digest(), 'EOS4yV91cqR6db8ntKtCLFVwWfvvvDMZctpd85GsaX5GUMaiqUWYL')


'SIG_K1_Kg3bwKkiPHu71aCM8dYvwZLn8pxGWWu4uZ9Zv5J8JQtsWmsnz5bMpuLbUNh4T4KtDVxWvKVMoRBFzmEvuqPGLzhAQz1rAP'

### sign with private key of EOS5gNqDpWm72KzNWFVgBDmybrEtk9qx2ZuAeRbK9iSJMVJ2B8y73

In [18]:
public_keys = ['EOS5gNqDpWm72KzNWFVgBDmybrEtk9qx2ZuAeRbK9iSJMVJ2B8y73']
info = uuosapi.get_info()
signed_trx = wallet.sign_transaction(signed_trx, public_keys, info['chain_id'])
sign2 = json.loads(signed_trx)['signatures']
sign2

['SIG_K1_Kg3bwKkiPHu71aCM8dYvwZLn8pxGWWu4uZ9Zv5J8JQtsWmsnz5bMpuLbUNh4T4KtDVxWvKVMoRBFzmEvuqPGLzhAQz1rAP',
 'SIG_K1_KhEERQjBU6qxZg19ZRVR2ztBaGtKjqQW37aDsP8h8cNX9y6R5kfwj6B2pQpcRPV4AAihiaEpN1h41rMCxjLsN15r6Mgzeq']

In [19]:
r = uuosapi.push_transaction(signed_trx, compress=False)
print(r['processed']['action_traces'][0]['console'])

hello,world

